In [1]:
!pip install -q langchain langchain-community langchain-google-genai google-generativeai pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.9 MB/s eta 0:00:00


In [15]:
from langchain_core.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)
from langchain_core.messages import SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers.pydantic import PydanticOutputParser
from pydantic import BaseModel, Field
import os
import getpass
import json

In [3]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your GOOGLE API Key: ")

Enter your GOOGLE API Key: ··········


In [4]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")  # Use 'gemini-pro' as the model

In [6]:
user_prompt_template = PromptTemplate(
    template = """
    Topic: {topic}
    Content: This is a content for the above topic:
    """,
    input_variables = ["topic"]
)

user_message_template = HumanMessagePromptTemplate(prompt=user_prompt_template)

In [7]:
content_chat_prompt_template = ChatPromptTemplate([
    SystemMessage(content="You are a content creator. Given the topic, it is your job to write short Instagram reel content, which can be spoken in 30 seconds."),
    user_message_template
])

In [8]:
content_chain = content_chat_prompt_template | model | StrOutputParser()

In [9]:
hashtag_prompt_template = PromptTemplate(
    template = """
    Content:
    {content}
    Generate Hash tags for the above content:
    """,
    input_varaiables = ["cotent"]
)

hashtag_message_template = HumanMessagePromptTemplate(prompt=hashtag_prompt_template)

In [10]:
class ContentOutputStructure(BaseModel):
  content: str = Field(description="Generated content")
  hashtags: str = Field(description="Generated hashtags")

parser = PydanticOutputParser(pydantic_object=ContentOutputStructure)

In [11]:
hashtag_system_prompt_template = PromptTemplate(
    template= """\n{format_instructions}\n You are a content creator. Given the content, it is your job to write hashtags for instagram reels.""",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

hashtag_system_message_template = SystemMessagePromptTemplate(prompt=hashtag_system_prompt_template)

In [12]:
hashtag_chat_prompt_template = ChatPromptTemplate.from_messages([
    hashtag_system_message_template,
    user_message_template
])

In [13]:
hashtags_chain = hashtag_chat_prompt_template | model | parser

In [14]:
overall_chain = content_chain | hashtags_chain

In [16]:
response = overall_chain.invoke({"topic": "Greatest snipers of all time"})

In [17]:
json_output = response.json()
output_dict = json.loads(json_output)

<ipython-input-17-e87feca780c5>:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  json_output = response.json()


In [18]:
print(f"JSON response: {output_dict}")

JSON response: {'content': 'Dive into the world of legendary snipers! From Simo Häyhä, the White Death, to Chris Kyle, the American Sniper, discover the stories behind these masters of precision and camouflage.  Who do YOU think is the greatest sniper of all time? Let us know in the comments!', 'hashtags': '#sniper #history #military #sharpshooter #marksman #legend #ww2 #warfare #covertwarfare #longrangeshooting #guns #rifle #precision #camouflage #simohayha #lyudmilapavlichenko #carloshathcock #chriskyle #americansniper #ladydeath #whitedeath #whitefeather #greatestsniper #top5 #commentbelow'}
